In [1]:
import deepchem as dc
import pandas as pd
from train_model import TOKENS, MAX_LENGTH
from rdkit import Chem
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2021-11-16 14:02:15.160728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-16 14:02:15.160751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Num GPUs Available:  0


2021-11-16 14:02:17.425005: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-16 14:02:17.425680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-16 14:02:17.425868: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-16 14:02:17.426017: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-16 14:02:17.426159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## Load the most recent checkpoint
I stored all checkpoints in `embeddings/seq2seq/data`

In [9]:
model = dc.models.SeqToSeq(TOKENS,
                           TOKENS,
                           MAX_LENGTH,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=256,
                           batch_size=100,
                           model_dir="data")

In [10]:
model.get_checkpoints()

['data/ckpt-43', 'data/ckpt-44', 'data/ckpt-45', 'data/ckpt-46', 'data/ckpt-47']

In [11]:
# loads the newest checkpoint
model.restore()

## Load all SMILES
and predict their embedding

In [12]:
canonicalize = lambda smile: Chem.MolToSmiles(Chem.MolFromSmiles(smile))
all_smiles = list(pd.read_csv("../lincs_trapnell.smiles")["smiles"].values)

In [13]:
# quick check on subset of all embeddings
pred = model.predict_from_sequences(all_smiles[0:15])
for s_pred, s_real in zip(pred, all_smiles[0:15]):
    s_pred = "".join(s_pred)
    print(f"{s_pred == s_real}\n-- {s_real}\n-- {s_pred}")

False
-- C[C@H](NC(=O)/C(C#N)=C/c1cccc(Br)n1)c1ccccc1
-- C[C@H](CS(S(=C(###=C(N/c1ccccBBr)n1)c1ccccc1
False
-- Cc1cc(Nc2cc(CN3CCOCC3)c3nc(C)c(Cc4ccc(Cl)cc4F)n3n2)[nH]n1
-- Cc1cc(Nc2cc(CN3CCCCCCCCc3ncCC)cccc4ccc(Cl)cc4C33)c[nH]])n1
False
-- Cc1cc(N2CCOCC2)cc2[nH]c(-c3c(NCC(O)c4cccc(Cl)c4)cc[nH]c3=O)nc12
-- Cc1cc(N2CC=CCN)ccccnN]c(cccc3CCCCCcccccccF)ccC))cc[[n]]==O)cc11
False
-- Cl.Cl.c1ccc([C@@H]2C[C@H]2NC2CCNCC2)cc1
-- C/C=C/c1ccc([CC@H]2[[C@H]2CN2CCNCCC))cc1
False
-- O=C(c1ccc(/C=C/c2n[nH]c3ccccc23)cc1)N1CCNCC1
-- O=C(c1ccc(/C=C/c2n[nH]c3ccccc32)cc1)N1CCCCC1
False
-- Cc1nnc(C(C)C)n1C1CC2CCC(C1)N2CCC(NC(=O)C1CCC(F)(F)CC1)c1ccccc1
-- Cc1nnc(C(C)C)C)C(CCCCCCCCCCCCCCCCCC(()))CCCCC((((CCCCC11ccccccc
False
-- NC(=O)c1ncn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c1N
-- N/(=O)c1cnc([C@@H]2O[C@H](OO)OC[@]](OO)[C]]]2O)c1
False
-- N#CCNC(=O)c1ccc(-c2ccnc(Nc3ccc(N4CCOCC4)cc3)n2)cc1
-- N#CCNC(=O)c1ccc(-c2cccc(Nc3ccc(N4CCOCC4)cc3)n2)cc1
False
-- C#Cc1cccc(Nc2ncnc3cc(OC)c(OCCCCCCC(=O)NO)cc23)c1
-- C#Cc1ccc

In [14]:
# actually predict all embeddings
emb = model.predict_embeddings(all_smiles)

## Store the resulting embedding

In [15]:
final_df = pd.DataFrame(emb, index=all_smiles, columns=[f"latent_{i+1}" for i in range(emb.shape[1])])

In [16]:
final_df.to_parquet("data/seq2seq.parquet")
final_df

,latent_1,latent_2,latent_3,latent_4,latent_5,latent_6,latent_7,latent_8,latent_9,latent_10,...,latent_247,latent_248,latent_249,latent_250,latent_251,latent_252,latent_253,latent_254,latent_255,latent_256
C[C@H](NC(=O)/C(C#N)=C/c1cccc(Br)n1)c1ccccc1,-0.015769,0.052912,-0.272413,0.305668,-0.110704,-0.145679,0.111034,0.435835,-0.160023,-0.182336,...,0.459639,0.198406,-0.335790,-0.142956,0.030977,0.018542,-0.009532,0.131064,-0.159807,0.067595
Cc1cc(Nc2cc(CN3CCOCC3)c3nc(C)c(Cc4ccc(Cl)cc4F)n3n2)[nH]n1,-0.007374,-0.198397,-0.492498,0.021952,-0.136411,0.262194,-0.089772,0.495753,-0.051472,-0.155910,...,0.528023,-0.030843,0.023274,0.432102,0.053639,-0.076816,0.045384,0.078301,-0.082535,-0.002804
Cc1cc(N2CCOCC2)cc2[nH]c(-c3c(NCC(O)c4cccc(Cl)c4)cc[nH]c3=O)nc12,0.061392,-0.164060,-0.591622,0.030527,-0.102847,0.087366,0.027974,0.524360,-0.053702,-0.127911,...,0.502180,-0.079729,0.129090,0.124276,0.006338,0.030390,0.091066,0.006866,0.267999,-0.159722
Cl.Cl.c1ccc([C@@H]2C[C@H]2NC2CCNCC2)cc1,-0.053740,0.177955,-0.367810,0.090018,0.058217,0.168071,-0.032419,0.498496,0.065081,0.061909,...,0.394874,0.083285,0.269693,0.023334,0.067825,-0.147118,-0.008010,0.101650,-0.354543,0.101166
O=C(c1ccc(/C=C/c2n[nH]c3ccccc23)cc1)N1CCNCC1,0.017594,0.015007,-0.387035,-0.025458,-0.049479,0.242852,0.069750,0.279869,-0.029466,-0.139046,...,0.418218,0.010910,-0.089801,0.030506,0.019540,-0.100496,0.032306,0.079607,0.224210,0.140878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCCC(=O)Nc1ccc2c(c1)C(=O)N(C)C[C@H](OC)[C@@H](C)CN(Cc1ccc(-c3ccccn3)cc1)[C@@H](C)CO2,-0.102524,-0.053212,-0.560887,0.181214,-0.010810,0.006538,-0.036625,0.648405,0.146833,-0.181918,...,0.556772,0.048353,0.037994,0.201654,-0.030352,-0.065399,0.072160,0.114033,-0.057180,-0.109707
Cc1cc(CS(=O)(=O)c2ccccc2)cc(OCc2ccc(CN3CCC[C@@H]3CO)cc2)c1,0.001217,-0.136761,-0.552015,-0.092697,-0.041933,0.018425,0.012212,0.603674,0.030974,0.016439,...,0.330128,0.196006,-0.098398,0.217345,-0.009975,-0.026851,-0.000996,0.012040,0.522048,-0.119071
CN(C)CCOc1ccc(/C(=C(\CCCl)c2ccccc2)c2ccccc2)cc1,-0.041137,-0.116007,-0.462455,-0.103874,0.030683,-0.179392,0.012681,0.526103,0.077992,-0.089742,...,0.422692,0.078122,0.289891,-0.157302,0.010337,-0.008215,0.097771,0.074439,-0.133566,0.171229
CC1(C)C=Cc2c(ccc3c2[N+]([O-])=C2C3=C[C@@]34NC(=O)[C@]5(CCCN5C3=O)C[C@H]4C2(C)C)O1,-0.053945,-0.123809,-0.651478,-0.041010,0.046580,0.001756,0.082595,0.700623,-0.193987,-0.100845,...,0.700816,0.102099,0.042866,0.031351,-0.002288,0.088115,0.113948,-0.051808,-0.131280,-0.060399


In [19]:
assert sorted(pd.read_csv("../lincs_trapnell.smiles")["smiles"].values) == sorted(final_df.index.values)